# Exercício 5 - 07-11-2024

## Estimação de Partitura

O sinal fornecido no arquivo sinal.mat é um pequeno trecho de música, gerado artificialmente. O objetivo deste trabalho é estimar a sequência de notas, com base nas seguintes informações:

1. O período de amostragem está na variável $T_a$ no arquivo;
1. A música foi tocada a um andamento de 80 notas por minuto
1. Em música, uma diferença de uma oitava corresponde a dobrar a frequência. No caso da gravação fornecida, o lá central (la3) corresponde a 442Hz. O próximo lá mais grave (la2) corresponde, portanto, a 221Hz, e o próximo mais agudo (la4), a 884Hz. Como há doze semitons em uma oitava (do-do#-re-re#-mi-fá-fá#-sol-sol#-lá-lá#-si-dó), subir um semitom equivale a multiplicar a frequência da nota por $\sqrt[12]{2} = 1.05946$
1. A menor diferença de frequências entre duas notas na música vai portanto corresponder à diferença entre a nota mais grave e o semitom acima. A nota mais grave no nosso caso é o sol2, que é um tom abaixo do la2, e portanto, correspondente à frequência de $f_0 = 221 / \sqrt[6]{2} = 196.8886$ Hz.

A próxima frequência (correspondente ao sol2#) é então $f_1 = f_0 \sqrt[12]{2} = 208.5962$Hz, e
a diferença é $\Delta f_{min} = f_1 - f_0 = 11.7076$ Hz.

Escolha o tipo de janela, e os valores de M, R e N para tentar identificar as notas da
melodia

# Instalação de dependências

In [1]:
#================================================#
# PSI 3432 - Processamento de Áudio e Imagem 
# Title : EP - Partitura																							
# Authors: Lucas Gaspar Mendonça, 
# Thiago da Rocha Calomino Gonçalves												
# Description:									
#================================================#

import Pkg

Pkg.add("Plots")
using Plots

Pkg.add("MAT")
using MAT

Pkg.add("LaTeXStrings")
using LaTeXStrings

Pkg.add("DSP")
using DSP

Pkg.add("FFTW")
using FFTW

Pkg.add("FindPeaks1D")
using FindPeaks1D

Pkg.add("WAV")
using WAV

   Updating registry at `~/.julia/registries/General`


   Updating 

git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...


   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


...

   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/home/thiago/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl", 162, UndefVarError(:HDF5Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007f77ddd72843, Ptr{Nothing} @0x00007f77dddff17c, Ptr{Nothing} @0x00007f77dddff73b, Ptr{Nothing} @0x00007f77dddff389, Ptr{Nothing} @0x00007f77dddff8f0, Ptr{Nothing} @0x00007f77dde009c8, Ptr{Nothing} @0x00007f77dde01616, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x00007f77dde19e08, Ptr{Nothing} @0x00007f77dde1a78e, Ptr{Nothing} @0x00007f77dde1a13a, Ptr{Nothing} @0x00007f77dde1a3c8, Ptr{Nothing} @0x00007f77dddf5105, Ptr{Nothing} @0x00007f77dde1b15d, Ptr{Nothing} @0x00007f77d00e5fa1, Ptr{Nothing} @0x00007f77b7d3af3c, Ptr{Nothing} @0x00007f77dddff75b, Ptr{Nothing} @0x00007f77dddff389, Ptr{Nothing} @0x00007f77dddff8f0, Ptr{Nothing} @0x00007f77dde009c8, Ptr{Nothing} @0x00007f77dde01616, Base.InterpreterIP i

 package versions...
   Updating

 `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving

 package versions...
   Updating

# Extração do sinal

In [2]:
#=Extração do sinal=#

#================================================#
# Parte 1
#================================================#

# sinal recebido
# cd("C:/Users/lucas/Documents/USP/Disciplinas/8_semestre/PES2/EC5")
cd("/home/thiago/Documents/2024-2/PSI3432/exerc5_musica")

arquivo = matread("sinal.mat")
sinal = arquivo["x"]
Ta = arquivo["Ta"]
fa = 1/Ta
n = 0:(length(sinal[:, 1])-1)

# Gráfico do sinal
# plot(sinal[20000:1:22000])

0:120022

In [ ]:
#= Tocar o áudio do sinal=#
# wavplay(sinal, fa)

# Parâmetros de janela

In [3]:
##== Parametros de janela ==##
f_min = 196.8886 # sol2
Δf_min = f_min*(2.0^(1/12) - 1) 

# N
N_min = Int(round(fa/Δf_min))
N = nextpow(2, N_min)

# M
M_max = (60/320)*fa

# Tomando-se a Janela de Kaiser de M pontos:
# p/ atenuação em lóbulo lateral de 90 dB
Δω = f_min/fa
M_min = Int(round((24π*(120+12))/(155*2π*Δω)+1))
M = Int(round((M_min+M_max)/2))
Asl = (60+120)/2

# Tomando uma janela de Hamming
#sinal_capado = sinal[24:120023]
if Asl < 13.26
    β = 0
elseif Asl < 60
    β = 0.76609(Asl-13.26)^(0.4)+0.09834(Asl-13.26)
elseif Asl < 120
    β = 0.12438(Asl+6.3)
end

wk = kaiser(M, β/π)

m_window_signal = []

start_idx = 118793
end_idx = length(sinal)
window_length = 1916


if (end_idx - start_idx) + 1 < window_length
    # print("oh")
    m_window_signal = vcat(sinal[start_idx:length(sinal)], zeros(window_length -( (end_idx - start_idx) + 1)))
    print(length(m_window_signal))
else
    # print("ok")
    m_window_signal = sinal[start_idx:end_idx]
end

1916

# Cálculo da STFT

In [4]:
# Função para calcular a STFT
function stft(signal, window, nfft)
    # Tamanho do sinal e janela
    signal_length = length(signal)
    window_length = length(window)
    
    # Número de janelas
    num_windows = 1 + div(signal_length, window_length)
    
    # Matriz para armazenar o resultado da STFT
    stft_matrix = Matrix{ComplexF64}(undef, nfft, num_windows)
    
    # Iterar sobre o sinal com deslocamentos de hop_size
    for i in 1:num_windows
        # Índices do segmento
        start_idx = (i - 1) * window_length + 1
        end_idx = start_idx + window_length - 1
        
        if end_idx > signal_length
            # print("oh")
            m_window_signal = vcat(sinal[start_idx:length(sinal)], zeros(end_idx - signal_length))
            print(length(m_window_signal))
        else
            # print("ok")
            m_window_signal = sinal[start_idx:end_idx]
        end
        # Extraindo o segmento do sinal
        segment = (m_window_signal .* window)
        padded_segment = vcat(segment, zeros(nfft - window_length))

        # Calculando a TDF do segmento

        spectrum = fft(padded_segment)
        
        if (i == 0)
            print(spectrum)
        end
        # Adicionando à matriz STFT
        stft_matrix[:, i] = spectrum
    end
    
    return hcat(stft_matrix...)  # Organizar resultados em colunas
end

stft (generic function with 1 method)

In [ ]:
nfft = 2048 # Tamanho da FFT para cada segmento
omega0 = 2π/nfft
windowaxis = map((k) -> k * omega0, 1:nfft+1)

# Calculando a STFT
stft_result = stft(sinal, wk, nfft)[0]
abs_stft_result = abs.(stft_result)
plot(windowaxis, abs_stft_result)


# Visualizando o espectrograma
# heatmap(windowaxis, abs_stft_result, xlabel="Time Frames", ylabel="Frequency Bins", title="STFT Magnitude Spectrum")

# Identificação de notas